In [ ]:
# Spark Session
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("Distributed Shared Variables")
    # .master("spark://192.168.1.12:7077")
    .config("spark.cores.max", 16)
    .config("spark.executor.cores", 4)
    .config("spark.executor.memory", "512M")
    .getOrCreate()
)

spark

In [ ]:
# # Spark Session
# from pyspark.sql import SparkSession

# spark = (
#     SparkSession
#     .builder
#     .appName("Understand Caching")
#     .master("local[*]")
#     .config("spark.executor.memory", "512M")
#     .getOrCreate()
# )

# spark

In [ ]:
# Read EMP CSV data

_schema = "first_name string, last_name string, job_title string, dob string, email string, phone string, salary double, department_id int"

emp = spark.read.format("csv").schema(_schema).option("header", True).load(r"/content/employee_records.csv")

In [ ]:
# Variable (Lookup)
dept_names = {1 : 'Department 1',
              2 : 'Department 2',
              3 : 'Department 3',
              4 : 'Department 4',
              5 : 'Department 5',
              6 : 'Department 6',
              7 : 'Department 7',
              8 : 'Department 8',
              9 : 'Department 9',
              10 : 'Department 10'}

In [ ]:
# Broadcast the variable

broadcast_dept_names = spark.sparkContext.broadcast(dept_names)

In [ ]:
# Check the value of the variable
broadcast_dept_names.value

{1: 'Department 1',
 2: 'Department 2',
 3: 'Department 3',
 4: 'Department 4',
 5: 'Department 5',
 6: 'Department 6',
 7: 'Department 7',
 8: 'Department 8',
 9: 'Department 9',
 10: 'Department 10'}

In [ ]:
# Create UDF to return Department name

from pyspark.sql.functions import udf, col

@udf
def get_dept_names(dept_id):
    return broadcast_dept_names.value.get(dept_id)

In [ ]:
emp_final = emp.withColumn("dept_name", get_dept_names(col("department_id")))

In [ ]:
emp_final.show()

+----------+----------+--------------------+----------+--------------------+--------------------+--------+-------------+---------+
|first_name| last_name|           job_title|       dob|               email|               phone|  salary|department_id|dept_name|
+----------+----------+--------------------+----------+--------------------+--------------------+--------+-------------+---------+
|   Richard|  Morrison|Public relations ...|1973-05-05|melissagarcia@exa...|       (699)525-4827|512653.0|         NULL|     NULL|
|     Bobby|  Mccarthy|   Barrister's clerk|1974-04-25|   llara@example.net|  (750)846-1602x7458|999836.0|         NULL|     NULL|
|    Dennis|    Norman|Land/geomatics su...|1990-06-24| jturner@example.net|    873.820.0518x825|131900.0|         NULL|     NULL|
|      John|    Monroe|        Retail buyer|1968-06-16|  erik33@example.net|    820-813-0557x624|485506.0|         NULL|     NULL|
|  Michelle|   Elliott|      Air cabin crew|1975-03-31|tiffanyjohnston@e...|       

In [ ]:
# Calculate total salary of Department 6

from pyspark.sql.functions import sum

emp.where("department_id = 6").groupBy("department_id").agg(sum("salary").cast("long")).show()

+-------------+---------------------------+
|department_id|CAST(sum(salary) AS BIGINT)|
+-------------+---------------------------+
+-------------+---------------------------+



In [ ]:
# Accumulators

dept_sal = spark.sparkContext.accumulator(0)

In [ ]:
# Use foreach

def calculate_salary(department_id, salary):
    if department_id == 6:
        dept_sal.add(salary)

emp.foreach(lambda row : calculate_salary(row.department_id, row.salary))

In [ ]:
# View total value

dept_sal.value

0

In [ ]:
 # Stop Spark Session

spark.stop()